# 네이버 플레이스 블로그 리뷰 크롤링
#### 2020-05-10 (일) 이한울
---
- 2020-05-10 (일): 플레이스 블로그 리뷰 긁어오기 및 스레딩 적용 완료
---
#### 2020-05-11 (월) 이한울
---
- 스레딩 실행 창에서 driver 경로를 본인의 크롬 드라이버가 있는 경로로 설정 후 실행
- time.sleep을 넉넉하게 주어야 HTTP 에러를 최대한 회피할 수 있습니다.
---
#### 2020-05-14 (목) 이한울
---
- 구글 드라이브 > 팀프로젝트 폴더 > [00]...마리톡 > [03] 데이터 > [03] 네이버 플레이스 리뷰 크롤링 > **naver_place_count_total_2020-05-01_final.csv** 파일을 다운로드, 원하는 경로로 이동시킨 뒤 진행해야 합니다.

In [19]:
import  urllib.request
import selenium
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
from requests import ConnectionError, HTTPError, Timeout, TooManyRedirects
import urllib3

from pandas import Series, DataFrame
import pandas as pd
import numpy as np

from datetime import datetime
import time
import sys
import math
import re
from tqdm import tqdm, tqdm_notebook

import selenium.common.exceptions as sce
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

from queue import Queue
from multiprocessing.pool import ThreadPool
from threading import Thread
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

import glob
import fnmatch

In [20]:
# 블로그 검색 결과를 저장할 DataFrame 생성
df_blog_review = \
DataFrame(columns=['store_id', 'store_name', 'title', 'writer', 'link', 'summary', 'blog_name', 'original_index'])

In [21]:
df_blog_error = DataFrame(columns=['store_id','store_name','error_code', 'original_index'])

In [22]:
df_blog_empty = DataFrame(columns=['store_id','store_name','original_index','blog_tab'])

In [23]:
# 블로그 본문을 저장할 빈 DataFrame 생성
df_text = DataFrame(columns=['store_id', 'store_name', 'link', 'main text', 'original_index'])

In [24]:
# 플레이스의 업소별 ID를 담은 csv파일 DataFrame으로 변환
# 본인이 naver_place_count_total_2020-05-01_final.csv를 저장한 경로로 설정
df_place = \
DataFrame(pd.read_csv('./naver_place_count_total_2020-05-01_final.csv', sep=',', header=0, index_col=0))
df_place

,store_id,store_id_only_num,store_name,booking,receipt,blog,total
0,s1518997092,1518997092,준오헤어 노원사거리점,2015.0,3.0,15.0,2033.0
1,s1571970153,1571970153,차홍룸 홍대점,2009.0,1.0,10.0,2020.0
2,s1142939643,1142939643,차홍룸 용산센트럴점,1871.0,3.0,11.0,1885.0
3,s1481443590,1481443590,차홍룸 용산점,1541.0,2.0,11.0,1554.0
4,s1177346002,1177346002,차홍룸 양재점,1373.0,1.0,10.0,1384.0
...,...,...,...,...,...,...,...
2722,s38257322,38257322,바바헤어 대치점,1.0,1.0,2.0,4.0
2723,s1069634711,1069634711,정환헤어샵,1.0,1.0,2.0,4.0
2724,s20852762,20852762,머리짱헤어샵,1.0,2.0,1.0,4.0
2725,s36091089,36091089,살롱160,1.0,1.0,2.0,4.0


# 블로그 본문 크롤링 함수
## 위 블로그 리뷰 섬네일 크롤링 함수 모두 완료 후 진행해야 합니다!
#### 2020-05-13 (수) 이한울
---
- 헤드리스 적용
- 스레드 미구현
- **위 블로그 리뷰 섬네일 크롤링 함수 모두 완료 후 진행해야 합니다!**
---
#### 2020-05-14 (목) 이한울
---
- 스레드 구현 완료
- 본인이 저장한 CSV 파일의 경로와 이름명을 read_csv를 통해 DataFrame으로 변환한 뒤 실행하셔야 합니다.
- **반드시 위 리뷰 썸네일 크롤링 함수에서 본인의 범위까지 크롤링을 완료한 뒤 진행해야 합니다.**
- **함수가 다 실행된 후 CSV파일로 별도로 저장해야 합니다. (함수화 하지 않음)**

In [25]:
# 블로그리뷰 섬네일 저장 CSV 파일 읽어오기
# 경로 직접 설정하셔야 합니다.
csv_path = './2020-05-20_blog_no_text(not-in 0-362) (10004-15005)FINAL.csv'
df_blog_review = pd.read_csv(csv_path, sep=',', header=0, index_col=0)
df_blog_review

,store_id,store_name,link,original_index
0,36364106,리소헤어 건대점,https://blog.naver.com/wjddusdn555/220560372452,110
1,36364106,리소헤어 건대점,https://blog.naver.com/e_bi_ni/221873699746,110
2,36364106,리소헤어 건대점,https://blog.naver.com/aamy78/220454696999,110
3,36364106,리소헤어 건대점,https://blog.naver.com/ggmpiano/220410099438,110
4,36364106,리소헤어 건대점,https://blog.naver.com/jjungvely89/220438956964,110
...,...,...,...,...
3806,82379273,라트리헤어 스파힐 강남베르점,https://blog.naver.com/pertty6265/221755319664,174
3807,82379273,라트리헤어 스파힐 강남베르점,https://blog.naver.com/wldnjs2465/221767297766,174
3808,82379273,라트리헤어 스파힐 강남베르점,https://blog.naver.com/wjddbfl0000/221742523594,174
3809,82379273,라트리헤어 스파힐 강남베르점,https://blog.naver.com/xowo_644/221852428504,174


In [26]:
# 에러가 발생한 블로그를 저장할 DataFrame 생성하기
df_text_error = DataFrame(columns=['store_id','store_name','link','original_index'])

In [27]:
# requests status code가 200이 아닌 것들을 저장할 DataFrame 생성하기
df_requests_error = DataFrame(columns=['store_id', 'store_name', 'error_code', 'link', 'original_index'])

In [28]:
df_no_text = DataFrame(columns=['store_id','store_name','link','original_index'])

In [29]:
def sub_emoji(content):
    
    find_txt = (content.text).strip().replace('\n', '')
    p = re.compile(r'[\\u\w]{1,}')
    filtered_txt = p.findall(find_txt)
    main_txt = ' '.join(filtered_txt)
    
    return main_txt

In [30]:
def main_text_save(main_txt, store_id, store_name, url):
    
    global df_text
    global df_place
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    
    df_text= df_text.append({
          'main text' : main_txt
        , 'store_id' : store_id
        , 'store_name' : store_name
        , 'link' : url
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_text

In [31]:
# 에러 발생 시 따로 저장할 함수 선언
def text_error_save(store_id, store_name, url):
    
    global df_text_error
    global df_place
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    
    df_text_error = df_text_error.append({
          'store_id' : store_id
        , 'store_name' : store_name
        , 'link' : url
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_text_error

In [32]:
def requests_error_save(store_id, store_name, error_code, url):
    
    global df_requests_error
    global df_place
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    
    df_requests_error = df_requests_error.append({
          'store_id' : store_id
        , 'store_name' : store_name
        , 'error_code' : error_code
        , 'link' : url
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_requests_error

In [33]:
def blog_no_text(store_id, store_name, url):
    
    global df_no_text
    global df_place
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    
    df_no_text = df_no_text.append({
          'store_id' : store_id
        , 'store_name' : store_name
        , 'link' : url
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_no_text   

In [34]:
# requests만 사용한 블로그 본문 크롤링 코드
# ==========================================================================
# 2020-05-12 (화) 이한울
# 본문을 긁어올 때에는 특별히 Action할 것이 없기 때문에
# requests를 통해서 크롤링하는 것이 효율적일 수도 있다고 생각하여 requests만 사용
# ==========================================================================

def text_save(url):
    
    global df_blog_review
    global df_text
    global df_place
    text_list = []
    headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"} 
    store_id = df_blog_review[df_blog_review['link'] == url]['store_id'].iloc[0]
    store_name = df_blog_review[df_blog_review['link'] == url]['store_name'].iloc[0]
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    url = url.replace('m.', '')
    time.sleep(1 + np.random.rand() + np.random.rand())
    
    print('\n', store_name, '의', url, '블로그의 본문 크롤링 중...\n')
    req = requests.get(url, headers)
    req_status = req.status_code
    time.sleep(1 + np.random.rand() + np.random.rand())
    
    if 'https://post.naver.com' in url:
        print('네이버 포스트 리뷰입니다. 크롤링에서 제외하고 다음 블로그를 크롤링합니다.')
        pass
    
    else:
        if req_status == 200:

            try:
                soup = BeautifulSoup(req.text, 'html.parser')
                bsb_main = soup.select('#mainFrame')

                if bsb_main:
                    for ii, blog in tqdm_notebook(enumerate(bsb_main)):
                        b_post = blog.attrs['src']
                        open_post = urlopen('http://blog.naver.com/' + b_post)
                        time.sleep(2.5)

                        b_html = BeautifulSoup(open_post, 'html.parser')
                        container_dic = {"class": "se-main-container"}
                        postview_dic = {"id": "postViewArea"}
                        wrap_dic = {"class": "se_component_wrap sect_dsc __se_component_area"}
                        
                        if b_html.find("div", container_dic):
                            txt_dic = container_dic

                        elif b_html.find("div", postview_dic):
                            txt_dic = postview_dic

                        elif b_html.find("div", wrap_dic):
                            txt_dic = wrap_dic

                        else:
                            blog_no_text(store_id, store_name, url)
                            print('잘못된 접근이거나 삭제된 게시물입니다.')
                            print('다음 블로그 크롤링을 진행합니다.')
                            break
                                
                        time.sleep(1)
                        content = b_html.find("div", txt_dic)
                        main_txt = sub_emoji(content)
                        main_text_save(main_txt, store_id, store_name, url)
                        print(url, '블로그의 본문 크롤링 성공!\n')


                else:
                    print('HTTP 에러 코드:',req.status_code)
                    text_error_save(store_id, store_name, url)

            except sce.NoSuchElementException:
                print('본문 크롤링 도중 에러가 발생했습니다.')
                text_error_save(store_id, store_name, url)

            except sce.NoSuchAttributeException:
                print('본문 크롤링 도중 에러가 발생했습니다.')
                text_error_save(store_id, store_name, url)

            except sce.ElementNotSelectableException:
                print('본문 크롤링 도중 에러가 발생했습니다.')
                text_error_save(store_id, store_name, url)

            except sce.WebDriverException:
                print('웹 드라이버 에러 발생')
                print('유니코드 변환 에러')
                text_error_save(store, store_name, url)
                
            except AttributeError:
                print('Attribute Error!')
                print('Error Code:', req_status)
                print('삭제된 게시물입니다.')
                
                blog_no_text(store_id, store_name, url)
                
        elif req_status in list(range(400, 506)):
            print('Requests Error가 발생했습니다. df_requests_error에', store_name + '의 정보를 저장합니다.')
            requests_error_save(store_id, store_name, req.status_code, url)
            pass
        
    print('현재까지 저장된 본문의 개수: ',len(df_text), '\n')
    time.sleep(1.25 + np.random.rand() + np.random.rand())
    
    
    req.close()
    return df_text

# 업소별 블로그 리뷰 본문 텍스트 크롤링 스레딩
#### 2020-05-14 (목) 이한울
---
- 앞선 플레이스 블로그 리뷰 썸네일 크롤링 함수에서 저장된 **df_blog_review** DataFrame이나 **CSV파일**로 따로 저장해두었다면 해당 CSV파일을 DataFrame으로 읽은 후 해당 DataFrame의 길이만큼 범위를 지정해주시면 됩니다.
- a와 end 값을 본인의 범위에 맞게 지정해주신 뒤 실행하시면 됩니다.
- 간혹 지정된 범위까지 크롤링을 하였음에도 계속해서 실행되어 뒷 부분까지 크롤링하는 상황이 발생하니 유의해주시기 바랍니다.
- **함수가 다 실행된 후 CSV파일로 별도로 저장해야 합니다. (함수화 하지 않음)**


# 만약 INDEX를 쪼개서 크롤링을 하신다면 반드시 반드시!! 아래 셀을 클릭한 뒤 Run All Above를 눌러서 df_text를 초기화 하셔야 합니다. 

## 안그러면 기존에 크롤링된 데이터가 그대로 남아있는 df_text에 append되어 나중에 duplicated() 때리시면 기절하실 수도 있습니다!

In [ ]:
# 플레이스 블로그 본문 크롤링 함수의 스레딩 적용
# 2020-05-15 (금) df_blog_review[0:10004]까지 크롤링
# 이후 10004부터 Iter하면 됨
# =========================================================================
# 약 800 ~ 1000개의 리뷰 크롤링 이후에 AttributeError가 자주 발생한다.
# 하지만 해당 링크로 접속했을 시 블로그 글에 정상적으로 접속할 수 있다,
# 이는 자동화 수집을 방지하기 위해 네이버에서 의도적으로 막은 것으로 보인다.
# 따라서 한번 수집 시 800 ~ 1000개를 넘지 않도록 해야한다.
# =========================================================================
a = 0 # 5759 #4583  # 4583 ~ 5564
start_no = 0 # 5759
end = a + 5
end_df = 3811 # 5759
# 2020-05-15 (금) 블로그 썸네일 csv에서 10000개 블로그 크롤링 진행중
ori_start_index = 0
ori_end_index = 362
# =========================================================================



for i in tqdm_notebook(range(int(math.ceil(end_df - start_no) / 5))):
    
    print(str(a) + ' ~ ' + str(end - 1) + '의 블로그 본문 크롤링을 시작합니다.')
    with ThreadPool(5) as tp:
        # 스레드에 입력할 매개변수가 2개 이상일 경우 starmap을 사용한다.
        tp.map(
            text_save
            , [df_blog_review['link'][ii] for ii in range(a, end)]
        )
        
    a += 5
    end += 5
    time.sleep(0.5)
    
    if a in range(100, end_df, 100):
        df_text.to_csv('./2020-05-20_place_blog-text 10004-15005(%i-%i) (%i-%i)TEMP.csv' %(ori_start_index, ori_end_index, start_no, end_df))
        df_text_error.to_csv('./2020-05-20_place_blog-error 10004-15005(%i-%i) (%i-%i)TEMP.csv' %(ori_start_index, ori_end_index, start_no, end_df))
        df_requests_error.to_csv('./2020-05-20_blog_requests-error 10004-15005(%i-%i) (%i-%i)TEMP.csv' %(ori_start_index, ori_end_index, start_no, end_df))
        df_no_text.to_csv('./2020-05-20_blog_no_text 10004-15005(%i-%i) (%i-%i)TEMP.csv' %(ori_start_index, ori_end_index, start_no, end_df))
    
    if a > end_df:
        df_text.to_csv('./2020-05-20_place_blog-text 10004-15005(%i-%i) (%i-%i)REAL.csv' %(ori_start_index, ori_end_index, start_no, end_df))
        df_text_error.to_csv('./2020-05-20_place_blog-error 10004-15005(%i-%i) (%i-%i)REAL.csv' %(ori_start_index, ori_end_index, start_no, end_df))
        df_requests_error.to_csv('./2020-05-20_blog_requests-error 10004-15005(%i-%i) (%i-%i)REAL.csv' %(ori_start_index, ori_end_index, start_no, end_df))
        df_no_text.to_csv('./2020-05-20_blog_no_text 10004-15005(%i-%i) (%i-%i)REAL.csv' %(ori_start_index, ori_end_index, start_no, end_df))
        break

# ==========================================================================
# 정상적으로 본문 크롤링이 된 블로그 정보 CSV 파일로 저장
df_text.to_csv('./2020-05-20_place_blog-text 10004-15005(%i-%i) (%i-%i)FINAL.csv' %(ori_start_index, ori_end_index, start_no, end_df))

# 크롤링 도중 에러가 발생한 블로그 정보 CSV 파일로 저장
df_text_error.to_csv('./2020-05-20_place_blog-error 10004-15005(%i-%i) (%i-%i)FINAL.csv' %(ori_start_index, ori_end_index, start_no, end_df))

# Requests Error가 발생한 블로그 정보 CSV 파일로 저장
df_requests_error.to_csv('./2020-05-20_blog_requests-error 10004-15005(%i-%i) (%i-%i)FINAL.csv' %(ori_start_index, ori_end_index, start_no, end_df))

# 본문이 없거나, Attribute Error가 발생한 블로그 정보 CSV파일로 저장
df_no_text.to_csv('./2020-05-20_blog_no_text 10004-15005(%i-%i) (%i-%i)FINAL.csv' %(ori_start_index, ori_end_index, start_no, end_df))
# ==========================================================================

0 ~ 4의 블로그 본문 크롤링을 시작합니다.


C:\Users\LEE-Bros PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: `item` has been deprecated and will be removed in a future version



 리소헤어 건대점 의 https://blog.naver.com/e_bi_ni/221873699746 블로그의 본문 크롤링 중...


 리소헤어 건대점 의 https://blog.naver.com/jjungvely89/220438956964 블로그의 본문 크롤링 중...


 리소헤어 건대점 의 https://blog.naver.com/ggmpiano/220410099438 블로그의 본문 크롤링 중...


 리소헤어 건대점 의 https://blog.naver.com/wjddusdn555/220560372452 블로그의 본문 크롤링 중...


 리소헤어 건대점 의 https://blog.naver.com/aamy78/220454696999 블로그의 본문 크롤링 중...



C:\Users\LEE-Bros PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: `item` has been deprecated and will be removed in a future version
  import sys


https://blog.naver.com/jjungvely89/220438956964 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  1 
https://blog.naver.com/e_bi_ni/221873699746
 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  2 

https://blog.naver.com/wjddusdn555/220560372452 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  3 

https://blog.naver.com/ggmpiano/220410099438 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  4 

https://blog.naver.com/aamy78/220454696999 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  5 

5 ~ 9의 블로그 본문 크롤링을 시작합니다.

 리소헤어 건대점 의 https://blog.naver.com/ghkdwjdgus10/221814471171 블로그의 본문 크롤링 중...


 리소헤어 건대점 의 https://blog.naver.com/sunny_0225/221821813707 블로그의 본문 크롤링 중...


 리소헤어 건대점 의 https://blog.naver.com/lovely_sine/220401148076 블로그의 본문 크롤링 중...


 리소헤어 건대점 의 https://blog.naver.com/nowlove03301/220379437940 블로그의 본문 크롤링 중...


 리소헤어 건대점 의 https://blog.naver.com/uuuuuujin/220298825676 블로그의 본문 크롤링 중...



...

https://blog.naver.com/navybottle/221799234504 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  564 

https://blog.naver.com/did3296/221815525044 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  565 

https://blog.naver.com/see823/221742774299 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  566 

https://blog.naver.com/skylee0331/221786806500 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  567 

https://blog.naver.com/hmlee8402/221801178891 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  568 

580 ~ 584의 블로그 본문 크롤링을 시작합니다.

 준오헤어 노원문화의거리점 의 https://blog.naver.com/gustnrbgml/221872492255 블로그의 본문 크롤링 중...


 준오헤어 노원문화의거리점 의 https://blog.naver.com/angie0061/221802926298 블로그의 본문 크롤링 중...


 준오헤어 노원문화의거리점 의 https://blog.naver.com/jh1011002/221584275437 블로그의 본문 크롤링 중...


 준오헤어 노원문화의거리점 의 https://blog.naver.com/kiss0152/221884455121 블로그의 본문 크롤링 중...


 준오헤어 노원문화의거리점 의 https://blog.naver.com/dalkommama/221800926130 블로그의 본문 크롤링 중...



https://blog.naver.com/gustnrbgml/221872492255 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  569 

https://blog.naver.com/angie0061/221802926298 블로그의 본문 크롤링 성공!
https://blog.naver.com/jh1011002/221584275437
 
블로그의 본문 크롤링 성공!
현재까지 저장된 본문의 개수: 
 571
 현재까지 저장된 본문의 개수: 
 
571 

https://blog.naver.com/kiss0152/221884455121 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  572 

https://blog.naver.com/dalkommama/221800926130 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  573 

585 ~ 589의 블로그 본문 크롤링을 시작합니다.

 준오헤어 노원문화의거리점 의 https://blog.naver.com/96___/221561624877
  준오헤어 노원문화의거리점블로그의 본문 크롤링 중...
 
의 https://blog.naver.com/gustnrbgml/221860218329 블로그의 본문 크롤링 중...


 준오헤어 노원문화의거리점 의 https://blog.naver.com/popopbook/221767308104 블로그의 본문 크롤링 중...


 준오헤어 노원문화의거리점 의 https://blog.naver.com/jaeeun114/221723822906 블로그의 본문 크롤링 중...


 준오헤어 노원문화의거리점 의 https://blog.naver.com/hmlee_0212/221801262049 블로그의 본문 크롤링 중...



https://blog.naver.com/96___/221561624877 https://blog.naver.com/gustnrbgml/221860218329 블로그의 본문 크롤링 성공!

블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수: 
 현재까지 저장된 본문의 개수: 575  575
 


https://blog.naver.com/hmlee_0212/221801262049 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  576 

https://blog.naver.com/popopbook/221767308104 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  577 

https://blog.naver.com/jaeeun114/221723822906 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  578 

590 ~ 594의 블로그 본문 크롤링을 시작합니다.

 준오헤어 노원문화의거리점 의 https://blog.naver.com/anplace/221559584247 블로그의 본문 크롤링 중...


 준오헤어 노원문화의거리점 의 https://blog.naver.com/joahaepcs/221690020055 블로그의 본문 크롤링 중...


 준오헤어 노원문화의거리점 의 https://blog.naver.com/ji-sung916/221791451103 블로그의 본문 크롤링 중...


 준오헤어 노원문화의거리점 의 https://blog.naver.com/anplace/221553976859 블로그의 본문 크롤링 중...


 준오헤어 노원문화의거리점 의 https://blog.naver.com/dlwldls90/221770182087 블로그의 본문 크롤링 중...



https://blog.naver.com/ji-sung916/221791451103 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  579 

https://blog.naver.com/dlwldls90/221770182087 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  580 

https://blog.naver.com/anplace/221553976859 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  581 

https://blog.naver.com/anplace/221559584247 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  582 

https://blog.naver.com/joahaepcs/221690020055 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  583 

595 ~ 599의 블로그 본문 크롤링을 시작합니다.

 준오헤어 노원문화의거리점 의 https://blog.naver.com/woweh8585/221545934433 블로그의 본문 크롤링 중...



 
 준오헤어 노원문화의거리점 준오헤어 노원문화의거리점준오헤어 노원문화의거리점   의의의   https://blog.naver.com/gustnrbgml/221849887902https://blog.naver.com/hi_9/221558534335https://blog.naver.com/leeyong_jh/221568940014   블로그의 본문 크롤링 중...
블로그의 본문 크롤링 중...
블로그의 본문 크롤링 중...




 준오헤어 노원문화의거리점 의 https://blog.naver.com/ar0104/221558654021 블로그의 본문 크롤링 중...



https://blog.naver.com/woweh8585/221545934433 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  584 

https://blog.naver.com/gustnrbgml/221849887902 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  585 

https://blog.naver.com/hi_9/221558534335 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  586 

https://blog.naver.com/ar0104/221558654021 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  587 

https://blog.naver.com/leeyong_jh/221568940014 블로그의 본문 크롤링 성공!


현재까지 저장된 본문의 개수:  588 

600 ~ 604의 블로그 본문 크롤링을 시작합니다.

 준오헤어 노원문화의거리점 의 https://blog.naver.com/mingjju21/221571499653 블로그의 본문 크롤링 중...


 준오헤어 노원문화의거리점 의 https://blog.naver.com/yyyyyyy4444/221594706284 블로그의 본문 크롤링 중...


 준오헤어 노원문화의거리점 의 https://blog.naver.com/rktvmfk/221554543254 블로그의 본문 크롤링 중...


 준오헤어 노원문화의거리점 의 https://blog.naver.com/gustnrbgml/221637744774 블로그의 본문 크롤링 중...


 준오헤어 노원문화의거리점 의 https://blog.naver.com/werefamily/221572589864 블로그의 본문 크롤링 중...



In [19]:
a = 10004 # 5759 #4583  # 4583 ~ 5564
start_no = 10004 # 5759
end_df = int(len(df_blog_review[start_no:15004]))
end_df

5000

In [30]:
df_blog_review

,store_id,store_name,title,writer,link,summary,blog_name,original_index
0,1222327123,리안헤어 도봉한신점,리안헤어 도봉한신점[셋팅펌 잘하는 미용실~!],ehqhd001,https://m.blog.naver.com/ehqhd001/221962185495,NaN,리안헤어 도봉한신점,364
1,1222327123,리안헤어 도봉한신점,리안헤어 도봉한신점[셋팅펌 잘하는 미용실~!],ehqhd001,https://m.blog.naver.com/ehqhd001/221958456253,NaN,리안헤어 도봉한신점,364
2,1222327123,리안헤어 도봉한신점,리안헤어 도봉한신점[커드잘하는 미용실~!],ehqhd001,https://m.blog.naver.com/ehqhd001/221956986334,NaN,리안헤어 도봉한신점,364
3,1222327123,리안헤어 도봉한신점,리안헤어 도봉한신점[셋팅펌 잘하는미용실~!],ehqhd001,https://m.blog.naver.com/ehqhd001/221952021365,NaN,리안헤어 도봉한신점,364
4,1222327123,리안헤어 도봉한신점,리안헤어 도봉한신점 [남자파마 잘하는미용실~!],ehqhd001,https://m.blog.naver.com/ehqhd001/221939345007,NaN,리안헤어 도봉한신점,364
...,...,...,...,...,...,...,...,...
27080,17947654,가오다0927,펌하로 오셨지만 염색했습니다 훨씬 세련미가 보이네요,gga1015,https://m.blog.naver.com/gga1015/221672743609,머리가 엄청 많이 떠서 늘 다운펌과 볼륨 펌하시는데 이번에도 또 펌하신다고 하시네요...,가오다0927,767
27081,17947654,가오다0927,친구 따라 왔고요 예쁘게 바이올렛 염색하고 싶어요,gga1015,https://m.blog.naver.com/gga1015/221638563705,이번 고객님은 바이올렛으로 투톤 옴브레 스타일로 염색하셨습니다 베이스는 연보라로 깔...,가오다0927,767
27082,17947654,가오다0927,굵은 웨이브랑 C컬펌은 지겨워요 히피펌 해주세요,gga1015,https://m.blog.naver.com/gga1015/221675873689,맨날 굵은 웨이브나 c컬펌만 했어요 다른느낌으로 히피펌 하고 싶어요 하고 방문하신 ...,가오다0927,767
27083,17947654,가오다0927,친구결혼식 가시려고 펌하고 싶어 하셨지만 염색이 더욱 필요 하시네요,gga1015,https://m.blog.naver.com/gga1015/221668382026,이번 고객님 친구결혼식 가신다고 펌하로 오셨네요 근데 제가 볼때는 퍼머보다 염색이 ...,가오다0927,767
